In [ ]:
import logging

from projects.pay.payments.models import Payment
from projects.pay.payments.tasks import send_payment_receipt_emails

logger = logging.getLogger("crowdfund")

paymeny_emails = Payment.objects.filter(email_sent)
for payment in paymeny_emails:
    try:
        send_payment_receipt_emails.delay(payment.id)
        logger.info(f"{payment} email successfully sent")
    except:
        logger.warning("email failed to send")

In [ ]:
completed_payments = ContactEvent.objects.filter(
            contact_uid=self.uid, event_type="portal_payment_paid"
        )
if completed_payments:
    timedeltas = []
    for payment_event in completed_payments:
        payment_data = Payment.objects.filter(
            pk=payment_event.data.get("payment_id")
        ).values("transaction_id", "created_at")
        transaction_data = Payment.objects.filter(
            transaction=payment_data[0].get("transaction_id")
        ).values("transaction__completed_date")
        payment_created_at = payment_data[0].get("created_at")
        transaction_completed_at = transaction_data[0].get("transaction__completed_date")
        timedeltas.append(transaction_completed_at - payment_created_at)

    average_timedelta = sum(timedeltas, datetime.timedelta(0)) / len(timedeltas)

    return average_timedelta.total_seconds()

In [ ]:
from django.db.models import F


completed_payments = ContactEvent.objects.filter(
            contact_uid=self.uid, event_type="portal_payment_paid"
        )
average_timedelta = Payments.objects.filter(id__in=[event.data.get("payment_id") for event in completed_payments]).aggregate(Avg(F("transaction__completed_date") - F("created_at")))